In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Dataset loading

In [2]:
vehicles_dataset = pd.read_excel('Dataset_vehiculos.xlsx')
fasecolda_guide_dataset = pd.read_csv('Guia_CSV_331.csv')

### Data cleaning

In [3]:
vehicles_reference = vehicles_dataset[['REFERENCIA']]
vehicles_reference.loc[:,'REFERENCIA'] = vehicles_reference['REFERENCIA'].str.lower().fillna('')
print("*Vehicles Reference* \n")
vehicles_reference.head()

*Vehicles Reference* 



,REFERENCIA
0,[3] vibrant mt 1250cc 2ab abs aa r13
1,morning 1.1 at 1100cc aa
2,
3,jhr + mt 2700cc td 4x2 abs
4,spice 1.4 mt 1400cc 4p abs 2ab ct


In [4]:
fasecolda_references = fasecolda_guide_dataset[['Referencia1', 'Referencia2', 'Referencia3', 'Codigo']]
fasecolda_references_combined = fasecolda_references.melt(id_vars=['Codigo'], value_name='ReferenciaCombinada').drop(columns=['variable'])
fasecolda_references_combined.loc[:,'ReferenciaCombinada'] = fasecolda_references_combined['ReferenciaCombinada'].str.lower().fillna('')
print("*Fasecolda References Combined* \n")
fasecolda_references_combined.head()

*Fasecolda References Combined* 



,Codigo,ReferenciaCombinada
0,101001,2141
1,208003,wrangler
2,208004,wrangler
3,206001,eagle
4,301001,qute


### Semantic similarity processing
1. TF-IDF technique
2. Vectors for each words
3. Cosine similarity

In [5]:
vectorizer = TfidfVectorizer().fit(pd.concat([vehicles_reference['REFERENCIA'], fasecolda_references_combined['ReferenciaCombinada']]))
vehicles_tfidf = vectorizer.transform(vehicles_reference['REFERENCIA'])
fasecolda_tfidf = vectorizer.transform(fasecolda_references_combined['ReferenciaCombinada'])
similarity_matrix = cosine_similarity(vehicles_tfidf, fasecolda_tfidf)
matches = similarity_matrix.argmax(axis=1)
matched_codes = fasecolda_references_combined.iloc[matches].reset_index().set_index('index')['Codigo']
vehicles_dataset['Codigo_Fasecolda'] = matched_codes.values
vehicles_dataset.head()

,PLACA,MARCA,LINEA,REFERENCIA,Codigo_Fasecolda,Referencia_asignada
0,1,KIA,PICANTO,[3] VIBRANT MT 1250CC 2AB ABS AA R13,4601251,NaN
1,2,KIA,PICANTO,MORNING 1.1 AT 1100CC AA,3201294,NaN
2,3,CHEVROLET,N300,NaN,101001,NaN
3,4,JAC,HFC1035KN,JHR + MT 2700CC TD 4X2 ABS,11311041,NaN
4,5,KIA,RIO UB EX,SPICE 1.4 MT 1400CC 4P ABS 2AB CT,4601149,NaN


### Validating

In [7]:
valid_codes = set(fasecolda_guide_dataset['Codigo'].astype(str))
validation_results = []
for index, row in vehicles_dataset.iterrows():
    assigned_code = str(row['Codigo_Fasecolda'])
    if assigned_code in valid_codes:
        validation_results.append("Yes")
    else:
        validation_results.append("No")
vehicles_dataset['Validation'] = validation_results
vehicles_dataset.head()

,PLACA,MARCA,LINEA,REFERENCIA,Codigo_Fasecolda,Referencia_asignada,Validation
0,1,KIA,PICANTO,[3] VIBRANT MT 1250CC 2AB ABS AA R13,4601251,NaN,Yes
1,2,KIA,PICANTO,MORNING 1.1 AT 1100CC AA,3201294,NaN,Yes
2,3,CHEVROLET,N300,NaN,101001,NaN,Yes
3,4,JAC,HFC1035KN,JHR + MT 2700CC TD 4X2 ABS,11311041,NaN,Yes
4,5,KIA,RIO UB EX,SPICE 1.4 MT 1400CC 4P ABS 2AB CT,4601149,NaN,Yes


### Dataset updating

In [8]:
vehicles_dataset.to_excel('vehiculos_codigo_fasecolda.xlsx', index=False)